In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("player_mvp_stats.csv")

In [3]:
del stats["Unnamed: 0"]

In [4]:
# genuinely so confused what happened here, just deleting the duplicate row
stats[stats["Player"] == "Carnell Lake"]

,Player,Tm,Pos,Age,Games G,Games GS,Passing Cmp,Passing Att,Passing Yds,Passing TD,...,T,W-L%,PF,PA,PD,MoV,SoS,SRS,OSRS,DSRS
4307,Carnell Lake,BAL,DB,34.0,15.0,1.0,0.0,0.0,0.0,0.0,...,0,0.625,303,265,38,2.4,0.8,3.2,-0.8,4.0
20329,Carnell Lake,JAX,FS,32.0,16.0,16.0,0.0,0.0,0.0,0.0,...,0,0.875,396,217,179,11.2,-4.8,6.4,1.3,5.1
34610,Carnell Lake,PIT,SS,29.0,13.0,13.0,0.0,0.0,0.0,0.0,...,0,0.625,344,257,87,5.4,-0.3,5.2,0.0,5.1
34651,Carnell Lake,NaN,RCB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0.688,372,307,65,4.1,1.3,5.3,2.7,2.6
34652,Carnell Lake,PIT,RCB/SS,30.0,16.0,16.0,0.0,0.0,0.0,0.0,...,0,0.688,372,307,65,4.1,1.3,5.3,2.7,2.6
34697,Carnell Lake,PIT,LCB,31.0,16.0,16.0,0.0,0.0,0.0,0.0,...,0,0.438,263,303,-40,-2.5,-1.3,-3.8,-5.1,1.3


In [5]:
# remove row 34651, duplicate w/ row 34652
stats = stats.drop(34651)

In [6]:
# check for any null values
pd.isnull(stats).sum()

Player               0
Tm                   0
Pos                  0
Age                  0
Games G              0
Games GS             0
Passing Cmp          0
Passing Att          0
Passing Yds          0
Passing TD           0
Passing Int          0
Rushing Att          0
Rushing Yds          0
Rushing Y/A          0
Rushing TD           0
Receiving Tgt        0
Receiving Rec        0
Receiving Yds        0
Receiving Y/R        0
Receiving TD         0
Fumbles Fmb          0
Fumbles FL           0
Scoring TD           0
Scoring 2PM          0
Scoring 2PP          0
Year                 0
Int                  0
Sk                   0
Tackles Comb         0
Tackles Solo         0
Tackles Ast          0
Team                 0
Votes                0
Share            44644
W                    0
L                    0
T                    0
W-L%                 0
PF                   0
PA                   0
PD                   0
MoV                  0
SoS                  0
SRS        

In [7]:
# replace NaN's w/ 0's
stats = stats.fillna(0)

In [8]:
predictors = ['Passing Cmp', 'Passing Att', 'Passing Yds', 'Passing TD',
       'Passing Int', 'Rushing Att', 'Rushing Yds', 'Rushing Y/A',
       'Rushing TD', 'Receiving Tgt', 'Receiving Rec', 'Receiving Yds',
       'Receiving Y/R', 'Receiving TD', 'Fumbles Fmb', 'Fumbles FL',
       'Int', 'Sk', 'Tackles Comb', 'Tackles Solo', 'Tackles Ast', 'W-L%',
       'PF', 'PA', 'PD', 'MoV', 'SRS', 'OSRS']

In [9]:
train = stats[stats["Year"] < 2021]

In [10]:
test = stats[stats["Year"] == 2021]

In [11]:
# random forest creates series of decision trees & averages predictions from those trees
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators=100, random_state=1, min_samples_split=5)

In [12]:
reg.fit(train[predictors], train["Share"])

RandomForestRegressor(min_samples_split=5, random_state=1)

In [13]:
predictions = reg.predict(test[predictors])

In [14]:
projected_votes = pd.DataFrame(predictions * 50, columns=["Projected MVP Votes"], index=test.index).round()

In [15]:
predictions = pd.DataFrame(predictions, columns=["Probability"], index=test.index)

In [16]:
combination = pd.concat([test[["Player", "Share", "Votes"]], predictions, projected_votes], axis=1)

In [17]:
# 2021
combination.sort_values("Probability", ascending=False).head(10)

,Player,Share,Votes,Probability,Projected MVP Votes
6677,Josh Allen,0.00,0.0,0.358073,18.0
17480,Aaron Rodgers,0.78,39.0,0.243967,12.0
41867,Tom Brady,0.20,10.0,0.209518,10.0
13327,Dak Prescott,0.00,0.0,0.188047,9.0
20047,Jonathan Taylor,0.00,0.0,0.120821,6.0
23758,Matthew Stafford,0.00,0.0,0.086658,4.0
2611,Kyler Murray,0.00,0.0,0.045097,2.0
35818,T.J. Watt,0.00,0.0,0.025267,1.0
10740,Joe Burrow,0.00,0.0,0.021258,1.0
23338,Justin Herbert,0.00,0.0,0.014967,1.0


In [329]:
# 2022
combination.sort_values("Probability", ascending=False).head(10)

,Player,Share,Votes,Probability,Projected MVP Votes
6732,Josh Allen,0.02,1.0,0.207764,10.0
22843,Patrick Mahomes,0.96,48.0,0.124061,6.0
26491,Kirk Cousins,0.00,0.0,0.096013,5.0
10800,Joe Burrow,0.00,0.0,0.031006,2.0
34523,Jalen Hurts,0.02,1.0,0.028704,1.0
23398,Justin Herbert,0.00,0.0,0.025207,1.0
43231,Derrick Henry,0.00,0.0,0.025128,1.0
41920,Tom Brady,0.00,0.0,0.014590,1.0
1188,Roquan Smith,0.00,0.0,0.014139,1.0
30606,Daniel Jones,0.00,0.0,0.012702,1.0


In [319]:
# 2023
combination.sort_values("Probability", ascending=False).head(10)

,Player,Share,Votes,Probability,Projected MVP Votes
13439,Dak Prescott,0.00,0.0,0.104105,5.0
38245,Christian McCaffrey,0.00,0.0,0.100758,5.0
5411,Lamar Jackson,0.98,49.0,0.054218,3.0
6779,Josh Allen,0.02,1.0,0.048032,2.0
16231,Jared Goff,0.00,0.0,0.044240,2.0
38241,Brock Purdy,0.00,0.0,0.042576,2.0
25235,Tua Tagovailoa,0.00,0.0,0.036028,2.0
25230,Raheem Mostert,0.00,0.0,0.029705,1.0
34576,Jalen Hurts,0.00,0.0,0.029232,1.0
23869,Matthew Stafford,0.00,0.0,0.027996,1.0
